In [ ]:
# Data handling
import numpy as np
import pandas as pd

# Plotting
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

# Models
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_validate, train_test_split, GridSearchCV

In [9]:
path_features_train = "../raw_data/tox21_dense_train.csv.gz"
X = pd.read_csv(path_features_train).set_index("Unnamed: 0")

In [ ]:
path_targets_train = "../raw_data/tox21_labels_train.csv.gz"
y = pd.read_csv(path_targets_train).set_index("Unnamed: 0")

In [18]:
y_mmp = y[["SR.MMP"]].dropna()
mmp_train = X.merge(y_mmp, how="inner", on="Unnamed: 0")


In [23]:
mmp_train

X_mmp = mmp_train.drop(columns='SR.MMP')
y_mmp = mmp_train['SR.MMP']

# Baseline Model: Random Forest Regression

In [ ]:
tree_classsifier = DecisionTreeClassifier()

scoring = {'accuracy': 'accuracy',
           'recall': 'recall'}

max_depth = np.arange(100,401,1)
min_samples_split = np.arange(0,1025,25)
min_samples_leaf = np.arange(1,100,1)
max_leaf_nodes = np.arange(10, 101, 10)
criterion=['entropy','gini']

param_grid = {
    'max_depth': max_depth,
    'min_samples_split': min_samples_split,
    'min_samples_leaf': min_samples_leaf,
    'max_leaf_nodes': max_leaf_nodes,
    'criterion': criterion
}

grid_search = GridSearchCV(tree_classsifier, param_grid, cv=5, scoring='recall')
#cv = cross_validate(tree_classsifier, X_mmp, y_mmp, cv=5, scoring=scoring)
grid_search.fit(X_mmp, y_mmp)

In [ ]:
# Grid of parameters to search

# Grid Search
grid_search = GridSearchCV(tree, param_grid, cv=5, scoring='accuracy')


In [39]:
print(cv['test_recall'].mean())
print(cv['test_accuracy'].mean())

0.5169491525423728
0.8496999954478446
